# Set parametars and variables in this analysis
- grop_api_key
- max_token = 512
- root_folder = "/Users/presteddy56"
- source_folder = "resources"
- list_file = "doc_name.json"
- document_folder = "pdfs"
- query:str

In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:

import pdf_documentation
import contextual_retrieval
import vectores_langchain
import chain

In [10]:
pdfs =pdf_documentation.process()
pdfs.set_data("/Users/presteddy56","resources","doc_name.json")
pdfs.documents("pdfs")
crude_data =pdfs.full_documents
print(type(crude_data))

{'countries': ['India 2002', 'United States 2019', 'United Kingdom 2022', 'United Kingdom Pharmacology 2015', 'France biology 2019', 'Malaysia 2019', 'Europe 2023', 'United States for Veterans 2022', 'Australia 2022', 'Canada 2016'], 'files': ['India_2002.pdf', 'APA_2019.pdf', 'NICE_2022.pdf', 'BAP_Antidepressants_2015.pdf', 'France_biology_2019.pdf', 'Malaysia_2019.pdf', 'Europe_2023.pdf', 'United_States_Veterans_2022.pdf', 'Australia_2022.pdf', 'Canada_2016.pdf']}
<class 'dict'>


In [11]:
contextual_lists = contextual_retrieval.process()
contextual_lists.set_documents(crude_data)
contextual_lists.tokenized_splitter(512)

/Users/presteddy56/WhatStandard/contextual_retrieval.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '© 2017 Indian Journal of Psychiatry | Published by Wolters Kluwer ‑ Medknow S34ASSESSMENT AND EV ALUATION ( table-1 )
Management of depression involves comprehensive 
assessment and proper establishment of diagnosis. The 
assessment must be based on detailed history, physical 
examination and mental state examinations.History must be 
obtained from all sources, especially the family. The diagnosis 
must be recorded as per the current diagnostic criteria. 
Depression often presents with a combination of 
symptoms of depressed mood, loss of interest or pleasure, 
decreased energy and fatigue, reduced concentration 
and attention,  reduced self-esteem and self-confidence, 
ideas of guilt and unworthiness, bleak and pessimistic 
views of the future, ideas or acts of self-harm or suicide, 
disturbed sleep

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
def chain_processing(query:str,k1:float=1.5, b:float=0.75, percentile:float=0.95,temperature=0.1):
    retrieval_sample = contextual_retrieval.process()
    retrieval_sample.chunk_lists= contextual_lists.chunk_lists
    retrieval_sample.BM_25(query=query, k_1=k1,b_=b)
    retrieval_sample.somantic()
    retrieval_sample.WCscoring()
    retrieval_sample.finally_selecting(percentile = percentile)
    shaped_sample=retrieval_sample.contextual_retrieved
    vectores_sample=vectores_langchain.process()
    vectores_sample.clear()
    vectores_sample.making_vectores(shaped_sample)
    retriever_sample=vectores_sample.vectorstores.as_retriever()
    doc_sample = vectores_sample.documents
    chain_sample = chain.process()
    chain_sample.template()
    chain_sample.setting_retriever(retriever_sample)
    chain_sample.llm(temperature = temperature)
    chain_sample.chaining()
    chain_sample.run(query, doc_sample)
    return chain_sample.answer

In [17]:
import gradio as gr

title = "Quick reference fo guidelines for people with depression and clinician"
description = """<center> <img src='/Users/presteddy56/WhatStandard/image.png' alt = "logo" width="550"/></center>"""

demo = gr.Interface(
    title = title,
    description = description,
    fn = chain_processing,
    inputs = ["text",
              gr.Slider(value=1.5,minimum=0,maximum=2,step=0.1),
              gr.Slider(value=0.75, minimum=0,maximum=1,step=0.1),
              gr.Slider(value=0.95,minimum=0.75, maximum=1,step=0.001),
              gr.Slider(value=0.1, minimum=0, maximum=1,step=0.1)],
    outputs= "text",
    live = False,
    allow_flagging="never",
    theme= gr.themes.Soft()
)
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
